You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

Read in, merge, and clean the four datasets to make a single combined pandas DataFrame.

Create a single pandas DataFrame called employees_final containing:

Index: `employee_id`

Columns: `employee_first_name`, `employee_last_name`, `employee_country`, `employee_city`, `employee_street`, `employee_street_number`, `emergency_contact`, `emergency_contact_number`, `relationship`, `monthly_salary`, `team`, `title`, `office`, `office_country`, `office_city`, `office_street`, `office_street_number`.

Change any missing values in column names starting with office to the word "Remote".

In [10]:
import pandas as pd
# Start coding here...

### Read all files

In [11]:
# 1. empolyee_information:- 
emp_inf = pd.read_excel('datasets/employee_information.xlsx')
# 1.2 emergency_contacts
emrg_cntct = pd.read_excel('datasets/employee_information.xlsx', sheet_name=1, header=None) # I added the `header=None` cuz i found out that the dataset had no columns 
# 2. employee_roles:-
emp_role = pd.read_json('datasets/employee_roles.json',orient="index" ) # to read and rotate the dataframe
# 3.office_addresses.csv
ofc_adrs = pd.read_csv('datasets/office_addresses.csv')

### Print all col names 

In [12]:
print(f' \n employee Info :-\n {emp_inf.columns},\n \n employee_role:- \n {emp_role.columns},\n \n office_adresses:- \n {ofc_adrs.columns},\n emergency_contacts\n {emrg_cntct.columns}')

 
 employee Info :-
 Index(['employee_id', 'employee_last_name', 'employee_first_name',
       'employee_country', 'employee_city', 'employee_street',
       'employee_street_number'],
      dtype='object'),
 
 employee_role:- 
 Index(['title', 'monthly_salary', 'team'], dtype='object'),
 
 office_adresses:- 
 Index(['office', 'office_country', 'office_city', 'office_street',
       'office_street_number'],
      dtype='object'),
 emergency_contacts
 Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')


In [13]:
emrg_cntct

,0,1,2,3,4,5
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


### add correct col names to emrg_cntct

In [14]:
# lets now add the correct column names to the emrg_contct dataframe
emrg_cntct.columns =['employee_id','employee_last_name','employee_first_name','emergency_contact','emergency_contact_number','relationship']

# lets check it 
if list (emrg_cntct.columns) == ['employee_id','employee_last_name','employee_first_name','emergency_contact','emergency_contact_number','relationship']:
    print('good columns were added')
else:
    print('sad it did not go right')
   
emrg_cntct

good columns were added


,employee_id,employee_last_name,employee_first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


### Now lets print out list of columns i will be using in the employees_final df

In [15]:
columns = '`employee_first_name`, `employee_last_name`, `employee_country`, `employee_city`, `employee_street`, `employee_street_number`, `emergency_contact`, `emergency_contact_number`, `relationship`, `monthly_salary`, `team`, `title`, `office`, `office_country`, `office_city`, `office_street`, `office_street_number`'
columns = columns.replace("`","'")
print(columns)

'employee_first_name', 'employee_last_name', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number'


### Time to merge all the dataframes into a new Temp dataframe (employees)

In [16]:
# lets merge the data now 
# 1 Merge info with offices
employees = emp_inf.merge(ofc_adrs,left_on= 'employee_country',right_on='office_country',how='left')
# 2 Merge emp with roles
employees =employees.merge(emp_role,left_on='employee_id',right_on=emp_role.index)
# 3 Merge emp with contacts
employees =employees.merge(emrg_cntct,on='employee_id',suffixes=('','_y'))
employees

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number,title,monthly_salary,team,employee_last_name_y,employee_first_name_y,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,NaN,NaN,NaN,NaN,NaN,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


# Now all the data is merged together, i can subset it to create the final dataframe then i will fill data as desired

In [17]:
subset_col = ['employee_first_name', 'employee_last_name', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number']
employees_final = employees[subset_col]
office_columns = ['office', 'office_country', 'office_city', 'office_street', 'office_street_number']
employees_final[office_columns] = employees_final[office_columns].fillna('Remote')
employees_final = employees_final.set_index(employees.employee_id)

In [18]:
employees_final

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote


#                                                                                 Done !